# Fridge calibration

Plot all capacitance calibration data.

Please refer to individiual functions in this notebook for description.

In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
import yaml
from scipy.constants import e, h
import xarray as xr

from tgp.frequency_correction import (
    generate_mapping,
    output_current_closed,
    output_current_open,
)


def plot_fits(
    dataset_left: xr.Dataset,
    dataset_right: xr.Dataset,
    fridge_parameters: dict,
    state: Literal["closed", "open"] = "closed",
    title: str = "",
) -> None:
    """
    Plots the measured capacitance calibration data (real conductance: blue dots,
    imaginary conductance: orange dots)
    and the conductances expected for given fridge_parameters
    (real conductance: blue lines, imaginary conductance: orange lines) for a
    fully pinched off device (state = “closed”, sample conductance = 0 ) or fully
    open device (state = “open”, sample conductance -> infinite;
    conductance dominated by line resistances).

    By fitting to the measured data using the capacitances in the
    fridge_parameters as fit parameters, the fridge parameters are further used
    to generate a mapping between measured conductances and actual sample conductances
    in the main text, following the procedure described in main text appendix B.
    Note that in the fits, the drain capacitances are assumed to be a sum of the
    capacitances in the left and the right lines (justified by the fact that there
    are drain contacts on both sides of the Al drain, whereas the approximation is valid
    as the drain capacitance has a negligible impact on the conductance mapping).

    Parameters
    ----------
    dataset_left
        Conductance matrix measurement dataset as a function of
        left contact AC excitation frequency with all junctions assumed to
        be pinched off. The frequency is assumed to be a parameter called
        'freq_left', and the standard conductance matrix variables are
        expected to be measured.
    dataset_right
        Conductance matrix measurement dataset as a function of
        right contact AC excitation frequency with all junctions assumed to
        be pinched off. The frequency is assumed to be a parameter called
        'freq_right', and the standard conductance matrix variables are
        expected to be measured.
    fridge_parameters
        A dictionary assumed to contain all relevant
        parameters apart from the line capacitances.
    """

    xL = dataset_left.g_ll.freq_left.values
    y1L = dataset_left.g_ll.values
    y2L = dataset_left.g_rl.values

    xR = dataset_right.g_rr.freq_right.values
    y1R = dataset_right.g_rr.values
    y2R = dataset_right.g_lr.values

    fig, axs = plt.subplots(2, 2, figsize=(8, 6))

    sim_gll, sim_grl, sim_glr, sim_grr = simulated_conductances(
        xL, xR, fridge_parameters, state=state
    )

    # Local left
    axs[0, 0].plot(xL, np.real(y1L), ".", xL, np.imag(y1L), ".")
    axs[0, 0].plot(xL, np.real(sim_gll), xL, np.imag(sim_gll))

    axs[0, 0].set_xlabel("$f_L$ (Hz)")
    axs[0, 0].set_ylabel("$g_{LL}$ ($g_0$)")

    # Local right
    axs[1, 0].plot(xR, np.real(y1R), ".", xR, np.imag(y1R), ".")
    axs[1, 0].plot(xR, np.real(sim_grr), xR, np.imag(sim_grr))

    axs[1, 0].set_xlabel("$f_R$ (Hz)")
    axs[1, 0].set_ylabel("$g_{RR}$ ($g_0$)")

    # Nonlocal left
    axs[0, 1].plot(xL, np.real(y2L), ".", xL, np.imag(y2L), ".")
    axs[0, 1].plot(xL, np.real(sim_grl), xL, np.imag(sim_grl))

    axs[0, 1].set_xlabel("$f_L$ (Hz)")
    axs[0, 1].set_ylabel("$g_{RL}$ ($g_0$)")

    # Nonlocal right
    axs[1, 1].plot(xR, np.real(y2R), ".", xR, np.imag(y2R), ".")
    axs[1, 1].plot(xR, np.real(sim_glr), xR, np.imag(sim_glr))

    axs[1, 1].set_xlabel("$f_R$ (Hz)")
    axs[1, 1].set_ylabel("$g_{LR}$ ($g_0$)")
    if title:
        fig.suptitle(title, fontsize=16)
    fig.tight_layout()


def simulated_conductances(
    frequencies_left: np.ndarray,
    frequencies_right: np.ndarray,
    fridge_parameters: dict,
    state: Literal["closed", "open"] = "closed",
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Evaluates the expected conductance matrix values for given fridge_parameters
    and used frequency tones on left (frequencies_left) or right (frequencies_right)
    port for a fully pinched off device (state = “closed”, sample conductance = 0 )
    or fully open device (state = “open”, sample conductance -> infinite; conductance
    dominated by line resistances).
    """
    sim_gll = np.zeros(len(frequencies_left), dtype=complex)
    sim_grl = np.zeros(len(frequencies_left), dtype=complex)

    sim_glr = np.zeros(len(frequencies_right), dtype=complex)
    sim_grr = np.zeros(len(frequencies_right), dtype=complex)
    g0 = e**2 / h

    for i, frequency in enumerate(frequencies_left):
        M = generate_mapping(frequency, fridge_parameters)
        if state == "closed":
            I_l = output_current_closed(M, [1, 0])
        else:
            I_l = output_current_open(M, [1, 0])

        sim_gll[i] = I_l[0] / g0
        sim_grl[i] = I_l[1] / g0

    for i, frequency in enumerate(frequencies_right):
        M = generate_mapping(frequency, fridge_parameters)
        if state == "closed":
            I_r = output_current_closed(M, [0, 1])
        else:
            I_r = output_current_open(M, [0, 1])

        sim_glr[i] = I_r[0] / g0
        sim_grr[i] = I_r[1] / g0

    return sim_gll, sim_grl, sim_glr, sim_grr


In [ ]:
fridge_folder = Path("../data/fridge")
data_folder = fridge_folder / "calibration"
fridge_yamls = sorted(fridge_folder.glob("*.yaml"))

for fn in fridge_yamls:
    for state in ("closed", "open"):
        with fn.open() as f:
            fridge_parameters = yaml.safe_load(f)
        fridge_name = fn.stem
        fname_left = data_folder / f"{fridge_name}_{state}_left.nc"
        fname_right = data_folder / f"{fridge_name}_{state}_right.nc"
        if fname_left.exists() and fname_right.exists():
            dataset_left = xr.open_dataset(fname_left, engine="h5netcdf")
            dataset_right = xr.open_dataset(fname_right, engine="h5netcdf")
            plot_fits(dataset_left, dataset_right, fridge_parameters, state, title=f"{fridge_name} {state}")
            plt.show()